### Rexpresenting text as numbers
-  The ml model can only understand number or vectotrs
-  while dealing wirth text, the first thing to do is convert text into vectores (array of numbers) and process is called vectorization
- The are mainly threee ways to do this
    - one-hot encoding
    -  encode each word with a unique number
    -  word embeding
    


In [23]:
# Dowload the dataset
import tensorflow as tf
url = "https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz"

dataset = tf.keras.utils.get_file("aclImdb_v1.tar.gz" , url , untar=True, cache_dir='.',
                                  cache_subdir='')


In [24]:
dataset

'./aclImdb_v1_extracted'

In [26]:
import os
import shutil

dataset_dir = os.path.join(dataset, "aclImdb")

dataset_dir

'./aclImdb_v1_extracted/aclImdb'

In [30]:
train_dir = os.path.join(dataset_dir, "train")
train_dir

'./aclImdb_v1_extracted/aclImdb/train'

In [36]:
os.listdir(train_dir)

['urls_unsup.txt',
 'neg',
 'urls_pos.txt',
 'urls_neg.txt',
 'pos',
 'unsupBow.feat',
 'labeledBow.feat']

In [33]:
remove_dir = os.path.join(train_dir, "unsup")
remove_dir

'./aclImdb_v1_extracted/aclImdb/train/unsup'

In [34]:
import shutil

shutil.rmtree(remove_dir)

In [38]:
# create dataset from bthe dir
batch_size =  1024
seed = 42

train_ds = tf.keras.utils.text_dataset_from_directory(train_dir, batch_size=batch_size , validation_split=0.2, subset="training", seed=seed)
val_ds  = tf.keras.utils.text_dataset_from_directory(train_dir, batch_size=batch_size, validation_split=0.2, subset="validation", seed=seed)

train_ds

Found 25000 files belonging to 2 classes.
Using 20000 files for training.
Found 25000 files belonging to 2 classes.
Using 5000 files for validation.


<_PrefetchDataset element_spec=(TensorSpec(shape=(None,), dtype=tf.string, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>

In [52]:
# configure datset for perfomrance
AUTO_TUNE =tf.data.AUTOTUNE
AUTO_TUNE

-1

In [55]:
train_ds = train_ds.cache().prefetch(buffer_size = AUTO_TUNE)
val_ds = val_ds.cache().prefetch(buffer_size  = AUTO_TUNE)

In [56]:
train_ds

<_PrefetchDataset element_spec=(TensorSpec(shape=(None,), dtype=tf.string, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>

In [57]:
embedd_layer = tf.keras.layers.Embedding(1000 , 5)
embedd_layer


<Embedding name=embedding, built=False>

In [68]:
embedd_layer(tf.constant([1.0,2.0,3.0])).shape

TensorShape([3, 5])

In [67]:
result = embedd_layer(tf.constant([[0, 1, 2], [3, 4, 5]]))
result.shape

TensorShape([2, 3, 5])

In [72]:
# starndardization
import string
import re

def custom_standardization(input_data):
  lowercase = tf.strings.lower(input_data)
  stripped_html = tf.strings.regex_replace(lowercase, '<br />', ' ')
  return tf.strings.regex_replace(stripped_html,
                                  '[%s]' % re.escape(string.punctuation), '')

In [73]:
vocab_size = 10000
seq_length = 100

vectorized_layer = tf.keras.layers.TextVectorization(
    standardize = custom_standardization,
    max_tokens = vocab_size,
    output_mode =  "int",
    output_sequence_length=seq_length
    
)

In [75]:
# make text opnly ds
text_ds = train_ds.map(lambda x, y : x)

In [78]:
vectorized_layer.adapt(text_ds)

In [88]:
embedding_dim=16

model = tf.keras.Sequential([
  vectorized_layer,
  tf.keras.layers.Embedding(vocab_size, embedding_dim, name="embedding"),
  tf.keras.layers.GlobalAveragePooling1D(),
  tf.keras.layers. Dense(16, activation='relu'),
  tf.keras.layers.Dense(1)
])

In [89]:
# tensorboard callbacks

tensorboard_callbacks = tf.keras.callbacks.TensorBoard(log_dir="logs")

In [91]:
loss_fn = tf.keras.losses.BinaryCrossentropy(from_logits=True)
optimizer = tf.keras.optimizers.Adam()
metrics = ["accuracy"]


model.compile(loss=loss_fn, optimizer=optimizer, metrics=metrics)

In [104]:
model.fit(train_ds, validation_data=val_ds, callbacks=tensorboard_callbacks, epochs=15)

Epoch 1/15
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 29ms/step - accuracy: 0.9399 - loss: 0.1618 - val_accuracy: 0.8238 - val_loss: 0.4335
Epoch 2/15
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 29ms/step - accuracy: 0.9407 - loss: 0.1584 - val_accuracy: 0.8224 - val_loss: 0.4380
Epoch 3/15
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 30ms/step - accuracy: 0.9417 - loss: 0.1551 - val_accuracy: 0.8214 - val_loss: 0.4427
Epoch 4/15
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 31ms/step - accuracy: 0.9435 - loss: 0.1518 - val_accuracy: 0.8210 - val_loss: 0.4475
Epoch 5/15
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 31ms/step - accuracy: 0.9447 - loss: 0.1486 - val_accuracy: 0.8208 - val_loss: 0.4524
Epoch 6/15
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 29ms/step - accuracy: 0.9473 - loss: 0.1455 - val_accuracy: 0.8196 - val_loss: 0.4574
Epoch 7/15
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 29ms/step - accuracy: 0.9489 - loss: 0.1424 - val_accuracy: 0.8198 - val_loss: 0.4626
Epoch 8/15
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 29ms/step - accuracy: 0.9504 - loss: 0.1394 - val_accuracy: 0.8194 - v

In [93]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ text_vectorization              │ (None, 100)            │             0 │
│ (TextVectorization)             │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding (Embedding)           │ (None, 100, 16)        │       160,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling1d        │ (None, 16)             │             0 │
│ (GlobalAveragePooling1D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 16)             │           272 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            17 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 480,869 (1.83 MB)

 Trainable params: 160,289 (626.13 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 320,580 (1.22 MB)

In [100]:
# visualize the tensorbaord : load tebnsoerboard extension

%load_ext tensorboard


The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [105]:
%tensorboard --logdir logs

Reusing TensorBoard on port 6006 (pid 42648), started 0:01:35 ago. (Use '!kill 42648' to kill it.)

In [106]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ text_vectorization              │ (None, 100)            │             0 │
│ (TextVectorization)             │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding (Embedding)           │ (None, 100, 16)        │       160,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling1d        │ (None, 16)             │             0 │
│ (GlobalAveragePooling1D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 16)             │           272 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            17 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 480,869 (1.83 MB)

 Trainable params: 160,289 (626.13 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 320,580 (1.22 MB)

In [116]:
weights = model.get_layer("embedding").get_weights()[0]
vocab = vectorized_layer.get_vocabulary() 

In [118]:
import io
out_v = io.open('vectors.tsv', 'w', encoding='utf-8')
out_m = io.open('metadata.tsv', 'w', encoding='utf-8')

for index, word in enumerate(vocab):
  if index == 0:
    continue  # skip 0, it's padding.
  vec = weights[index]
  out_v.write('\t'.join([str(x) for x in vec]) + "\n")
  out_m.write(word + "\n")
out_v.close()
out_m.close()